# Collect Basic Data from zaubacorp ([ColabLink](https://colab.research.google.com/drive/1_dd3q87cpQmp9HS-OHIg2tMkI7vIXnq8#scrollTo=t2KigaPJpmKU))

In [ ]:
import requests
from tqdm.notebook import tqdm_notebook,tqdm
from IPython.display import display
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import numpy as np
import json
import copy
import re

In [ ]:
data = pd.read_csv('./data_/company_master_data_upto_Mar_2015_Telangana.csv',encoding='latin-1')
print("> Headers :",data.columns)
print("> Size :",data.shape)
data.head()

In [ ]:
count = 0
basic_url = 'https://www.zaubacorp.com/company/'
for i in tqdm_notebook(range(len(data['CORPORATE_IDENTIFICATION_NUMBER']))):
    comp_name = '-'.join(data['COMPANY_NAME'][i].split(' '))
    number = data['CORPORATE_IDENTIFICATION_NUMBER'][i]
    number='U'+number[1:]
    r = requests.get(basic_url+comp_name+'/'+number)
    html_data = pd.read_html(r.content)
    if(r.ok):
        count+=1

print(count)

In [ ]:
def display_all(data):
    for i in range(len(data)):
        print(i)
        display(data[i])

In [ ]:
# tables = ['Company Details','Previous Names','Previous CINS','Share Capital & Number of Employees',
# 'Listing and Annual Compliance Details','']
def get_basic_data(data,d_company):
    temp = np.array(data)
    for i in range(len(temp)):
        d_company[temp[i][0]]=temp[i][1]

def previous_cin(data,d_company):
    d_company['Previous CIN'] = (np.array(data)).tolist()

def previous_names(data,d_company):
    d_company['Previous Names'] = (np.array(data)).tolist()



In [ ]:
def fetch_data(link):
    r = requests.get(link)
    temp_data = pd.read_html(r.content,header=None)
    d_company = {}
    #NOTE Company Details
    get_basic_data(temp_data[0],d_company)

    #NOTE Share Capital & Number of Employees some data Available After Login
    get_basic_data(temp_data[3],d_company)

    #NOTE listing status
    get_basic_data(temp_data[4],d_company)

    #NOTE Balance Sheet
    get_basic_data(temp_data[5],d_company)

    #NOTE Profit & Loss
    get_basic_data(temp_data[6],d_company)

    #NOTE Previous Company Names available After login
    previous_cin(temp_data[1],d_company)

    #NOTE Previous Company CINs available After login
    previous_names(temp_data[1],d_company)

    # print(json.dumps(d_company,indent=5))
    # display_all(temp_data)
    return d_company

In [ ]:
json_file = fetch_data("https://www.zaubacorp.com/company/RUHIKA-INFRAVENTURES-LLP/AAR-2517")
print(len(json_file))
print(json.dumps(json_file,indent=5))

In [ ]:
with open('../Data-Collector/data/zaubacorp.com/links.json') as jsonfp:
    company_links = json.load(jsonfp)
    
print("Length of Data :",len(company_links))
count = 0
company_data = {}
with tqdm_notebook(total=len(company_links)) as pbar:
    for i in company_links:
        json_data = fetch_data(company_links[i]['link'])
        company_data[i] = json_data
        pbar.update()